# llama_3.1 8B Instruct

This model could ben downloaded and deploy locally, despite its slow runtime. 

In [ ]:
import torch
import transformers

In [ ]:
model_id = "./Llama-3.1-8B-Instruct" # local path to the model

pipeline: transformers.pipeline = transformers.pipeline("text-generation", model=model_id, device_map="auto", model_kwargs={"torch_dtype": torch.float16, "low_cpu_mem_usage": True})
terminators = [pipeline.tokenizer.eos_token_id, pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")]

In [ ]:
# takes a str query and returns str responses
def get_response(query, message_history=[], max_tokens=2048, temperature=0.6, top_p=0.9):
    user_prompt = message_history + [{"role": "user", "content": query}]
    prompt = pipeline.tokenizer.apply_chat_template(
        user_prompt, tokenize=False, add_generation_prompt=True
    )
    outputs = pipeline(
        prompt,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p
    )
    response = outputs[0]["generated_text"][len(prompt):]
    return response

# take a list of messages (dict[str, any] -> role, content) and a response
def response_from_msg_list(msg_list: list[dict], message_history=[], max_tokens=2048, temperature=0.6, top_p=0.9):
    flat_msg_list = []
    for x in range(len(msg_list)): flat_msg_list += [{"role":k , "content": v} for k, v in msg_list[x].items()]
    user_prompt = message_history + flat_msg_list
    print("message list:", user_prompt)
    prompt = pipeline.tokenizer.apply_chat_template(
        user_prompt, tokenize=False, add_generation_prompt=True
    )
    outputs = pipeline(
        prompt,
        max_new_tokens=max_tokens,
        eos_token_id=terminators,
        do_sample=True,
        temperature=temperature,
        top_p=top_p
    )
    response = outputs[0]["generated_text"][len(prompt):]
    return response

In [ ]:
get_response("What is the capital of Canada in one word and what's your confidence as a percentage that this is the correct answer?")

In [ ]:
r = response_from_msg_list([{"user": "What is the most populated city in Australia?"}, {"system": "provide the answer as concisely as possible and your confidence as a percentage at the end"}])
r